In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
warnings.filterwarnings('ignore')

In [4]:
# 导入表1 role_info
role_info = pd.read_pickle('./role_info.pkl')

In [5]:
role_info.shape

(7095415, 29)

In [6]:
role_info.head()

,user_id,cp_server_no,cp_role_id,create_role_time,create_role_date,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,...,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_data_type,channel_id,source_id,media_id
0,88251996,660001,12900795,2020-07-09 12:59:06,2020-07-09,4.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0.0,0.0,1,639,100,0
1,98746144,660001,13567751,2020-07-08 02:03:21,2020-07-08,11.0,1.0,3323.0,1.0,2.0,...,0,0,0,0,0.0,0.0,1,20176,239118,32
2,103232223,660001,14570940,2020-06-30 17:03:38,2020-06-30,8.0,1.0,2.0,1.0,1.0,...,0,0,0,0,0.0,0.0,1,20541,247450,16
3,103742098,660001,14689939,2020-06-28 23:18:58,2020-06-28,1.0,1.0,332.0,5.0,1.0,...,0,0,0,0,0.0,0.0,1,6867,284945,16
4,104799007,660001,15015286,2020-07-07 21:21:50,2020-07-07,7.0,1.0,6420.0,22.0,2.0,...,0,0,0,0,0.0,0.0,1,7162,265833,32


In [7]:
# 导入表2 role_predict_info
role_predict_data = pd.read_pickle('./role_predict_data.pkl')

In [8]:
role_predict_data.shape

(7095415, 11)

In [9]:
role_predict_data.head()

,user_id,cp_server_no,cp_role_id,create_role_time,create_role_date,role_type,channel_id,source_id,media_id,role_data_type,predict_30_pay
109,116258787,660005,18597192,2020-07-05 06:22:57,2020-07-05,2,6867,322145,10,1,281.003862
134,111784938,660056,17284948,2020-07-15 22:15:36,2020-07-15,2,6770,204493,16,1,553.922431
138,116740806,660066,18738039,2020-07-10 21:03:55,2020-07-10,2,6867,322145,10,1,265.876459
331,103855656,660323,醉㐅Nick,2020-07-20 13:46:33,2020-07-20,2,20021,22791,0,1,769.729772
397,106139684,660425,15479096,2020-06-25 11:25:04,2020-06-25,2,20570,271925,152,1,658.325536


In [90]:
# 获取计划信息表,source
source_1 = pd.read_csv('./source_1.csv')
source_2 = pd.read_csv('./source_2.csv')
source_1.rename(columns={'chl_user_id':'channel_id'},inplace=True)
source_1.drop_duplicates(subset=['channel_id','source_id'],inplace=True)
source = pd.merge(source_1,source_2,on=['channel_id','source_id'],how='left',validate='one_to_many')
source.dropna(inplace=True)
source['create_time'] = pd.to_datetime(source['create_time']).dt.date
source['tdate'] = pd.to_datetime(source['tdate']).dt.date
source.rename(columns={'tdate':'source_run_date'},inplace=True)
source = source[source['amount']!=0]
groups = source.groupby(['channel_id','source_id'])
temp = groups['create_time'].agg([('first_amount_time','min')]).reset_index()
source = pd.merge(source,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')

In [91]:
# 导入source_info
data = pd.read_csv('./data.csv',index_col=0)

In [92]:
source.shape

(82842, 11)

In [101]:
source['source_run_date'] = pd.to_datetime(source['source_run_date'])

In [105]:
source.head()

,channel_id,source_id,create_time,source_run_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,first_amount_time
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,2020-01-02
1,20953,252731,2020-01-02,2020-01-03,0.71,0.71,6.0,3.0,1054.0,6.0,2020-01-02
2,20953,252732,2020-01-02,2020-01-02,2280.22,2280.22,3002.0,119.0,108335.0,117.0,2020-01-02
3,20953,252732,2020-01-02,2020-01-03,700.59,700.59,1049.0,45.0,34087.0,45.0,2020-01-02
4,20953,252733,2020-01-02,2020-01-02,2200.00,2200.00,666.0,219.0,53623.0,202.0,2020-01-02


In [94]:
data['channel_id'] = data['channel_id'].astype(np.int64)
data['source_id'] = data['source_id'].astype(np.int64)

In [49]:
data.head()

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,time,pay_sum
1,20540,323114,0.0,0.0,0.0,589.49,1,0.0
2,20540,323113,0.0,0.0,0.0,536.25,1,0.0
3,20953,323225,0.0,0.0,2.0,1071.33,1,0.0
4,21116,322970,0.0,0.0,1.0,523.95,1,0.0
5,20953,323072,0.0,0.0,1.0,500.96,1,0.0


In [95]:
for i in range(1,31):
    exec("data['time'] = data['time'].replace(i,'2020-07-%s')"% (i))
data['time'] = pd.to_datetime(data['time'])
data.rename(columns={'time':'source_run_date'},inplace=True)

In [104]:
data.head()

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum
1,20540,323114,0.0,0.0,0.0,589.49,2020-07-01,0.0
2,20540,323113,0.0,0.0,0.0,536.25,2020-07-01,0.0
3,20953,323225,0.0,0.0,2.0,1071.33,2020-07-01,0.0
4,21116,322970,0.0,0.0,1.0,523.95,2020-07-01,0.0
5,20953,323072,0.0,0.0,1.0,500.96,2020-07-01,0.0


In [97]:
data.shape

(44210, 8)

In [106]:
data = pd.merge(data,source,on=['channel_id','source_id','source_run_date'],how='left',validate='one_to_one')

In [108]:
data.dropna(how='any',inplace=True)

In [109]:
data

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount_x,source_run_date,pay_sum,create_time,amount_y,paper_use_amount,click_num,down_num,pv,create_role_num,first_amount_time
0,20540,323114,0.0,0.0,0.0,589.49,2020-07-01,0.0,2020-07-01,589.49,618.96,553.0,0.0,141243.0,0.0,2020-07-01
1,20540,323113,0.0,0.0,0.0,536.25,2020-07-01,0.0,2020-07-01,536.25,563.06,659.0,0.0,90302.0,0.0,2020-07-01
2,20953,323225,0.0,0.0,2.0,1071.33,2020-07-01,0.0,2020-07-01,1071.33,1124.90,380.0,2.0,106584.0,2.0,2020-07-01
3,21116,322970,0.0,0.0,1.0,523.95,2020-07-01,0.0,2020-06-30,523.95,550.15,389.0,1.0,25607.0,1.0,2020-06-30
4,20953,323072,0.0,0.0,1.0,500.96,2020-07-01,0.0,2020-07-01,500.96,526.01,368.0,1.0,28206.0,1.0,2020-07-01
